#### 一. DataSource
1. Flink 中你可以使用 `StreamExecutionEnvironment.addSource(sourceFunction)` 来为你的程序添加数据来源。
2. 自带DataSource分类   
   1. 基于集合
   2. 基于文件  
    ```readFile(fileInputFormat, path, watchType, interval, pathFilter, typeInfo) ```
        1. `fileInputFormat` 和`读取路径`负责读取文件。根据提供的 
        2. `watchType`: 
            * `FileProcessingMode.PROCESS_CONTINUOUSLY`: 表示这个 source 可以定期（每隔 interval 毫秒）监测给定路径的新数据，
            * `FileProcessingMode.PROCESS_ONCE`: 表示处理一次路径对应文件的数据并退出。
            * `pathFilter`: 进一步排除掉需要处理的文件。

        在具体实现上，Flink 把文件读取过程分为两个子任务，即`目录监控`和`数据读取`。每个子任务都由单独的实体实现。`目录监控`由单个非并行（并行度为1）的任务执行，而`数据读取`由并行运行的多个任务执行。后者的并行性等于作业的并行性。单个目录监控任务的作用是扫描目录（根据 watchType 定期扫描或仅扫描一次），查找要处理的文件并把文件分割成切分片（splits），然后将这些切分片分配给下游 reader。reader 负责读取数据。每个切分片只能由一个 reader 读取，但一个 reader 可以逐个读取多个切分片。  
        重要注意：

        如果 watchType 设置为 `FileProcessingMode.PROCESS_CONTINUOUSLY`，则当文件被修改时，其内容将被重新处理。这会打破“exactly-once”语义，因为在文件末尾附加数据将导致其所有内容被重新处理。

        如果 watchType 设置为 `FileProcessingMode.PROCESS_ONCE`，则 source 仅扫描路径一次然后退出，而不等待 reader 完成文件内容的读取。当然 reader 会继续阅读，直到读取所有的文件内容。关闭 source 后就不会再有检查点。这可能导致节点故障后的恢复速度较慢，因为该作业将从最后一个检查点恢复读取。  
   3. 基于 Socket

3. DataSink
4. Transformation操作
   1. `Map`
        ```java
        SingleOutputStreamOperator<Student> map = student.map(new MapFunction<Student, Student>() {
            @Override
            public Student map(Student value) throws Exception {
                Student s1 = new Student();
                s1.id = value.id;
                s1.name = value.name;
                s1.password = value.password;
                s1.age = value.age + 5;
                return s1;
                }
            });
        map.print();
        ```
    2. `KeyBy`:  
    KeyBy 在逻辑上是基于 key 对流进行分区。在内部，它使用 hash 函数对流进行分区。它返回 KeyedDataStream 数据流。
        ```java
        KeyedStream<Student, Integer> keyBy = student.keyBy(new KeySelector<Student, Integer>() {
            @Override
            public Integer getKey(Student value) throws Exception {
                return value.age;
            }
        });
        keyBy.print();
        ```
    3. Reduce
       * 执行 reduce 操作只能是 KeyedStream
       * Reduce 返回单个的结果值，并且 reduce 操作每处理一个元素总是创建一个新值。常用的方法有 average, sum, min, max, count，使用 reduce 方法都可实现。
        ```java
        SingleOutputStreamOperator<Student> reduce = student.keyBy(new KeySelector<Student, Integer>() {
            @Override
            public Integer getKey(Student value) throws Exception {
                return value.age;
            }
        }).reduce(new ReduceFunction<Student>() {
            @Override
            public Student reduce(Student value1, Student value2) throws Exception {
                Student student1 = new Student();
                student1.name = value1.name + value2.name;
                student1.id = (value1.id + value2.id) / 2;
                student1.password = value1.password + value2.password;
                student1.age = (value1.age + value2.age) / 2;
                return student1;
            }
        });
        reduce.print();
        ```
    4. `Window`  
       Window 函数允许按时间或其他条件对现有 KeyedStream 进行分组。 以下是以 10 秒的时间窗口聚合：
        ```java
        inputStream.keyBy(0).window(Time.seconds(10));
        ```
    5. `Window join`  
       将同一个 window 的两个不同数据流 join 起来。
       ```java
       // 以上示例是在 5 秒的窗口中连接两个流，其中第一个流的第一个属性的连接条件等于另一个流的第二个属性。
       inputStream.join(inputStream1)
            .where(0).equalTo(1)
            .window(Time.seconds(5))     
            .apply (new JoinFunction () {...});
       ```
    6. `Split`  
       split将流拆分为逻辑上的两个或多个流。通过标签
        ```java
        SplitStream<Integer> split = inputStream.split(new OutputSelector<Integer>() {
            @Override
            public Iterable<String> select(Integer value) {
                List<String> output = new ArrayList<String>(); 
                if (value % 2 == 0) {
                    output.add("even");
                }
                else {
                    output.add("odd");
                }
                return output;
            }
        });
        ```
    7. `Select`  
        `select`将从split后的流中选择特定流。
        ```java
        SplitStream<Integer> split;
        DataStream<Integer> even = split.select("even"); 
        DataStream<Integer> odd = split.select("odd"); 
        DataStream<Integer> all = split.select("even","odd");
        ```

#### 二. 什么是用于流处理的窗口
1. window出现的原因  
    通常来讲，Window 就是用来对一个无限的流设置一个有限的集合，在有界的数据集上进行操作的一种机制。window 又可以分为基于时间（**Time-based**）的 window 以及基于数量（**Count-based**）的 window。

2. **Time-based**窗口
   1. 滚动时间窗口(tumbling time window)
    窗口之间没有交叉, 适合处理每隔多长时间统计一次的语义
        ```java
        data.keyBy(1)
        .timeWindow(Time.minutes(1)) //tumbling time window 每分钟统计一次数量和
        .sum(1);
        ```

    2. 滑动时间窗口(sliding time windows)  
        * 窗口之间有交叉, 需要制定2个参数: 多长时间滑动一次, 一次滑动跨过多长时间
        * 适合表示多长时间计算一次, 一次计算多长时间的数据的语义
        ```java
        data.keyBy(1)
            .timeWindow(Time.minutes(1), Time.seconds(30)) //sliding time window 每隔 30s 统计过去一分钟的数量和
            .sum(1);
        ```

3. **Count-based**
   1. tumbling count window
        ```java
        data.keyBy(1)
                .countWindow(100) //统计每 100 个元素的数量之和
                .sum(1);
        ```
   2. sliding count window
        ```java
        data.keyBy(1) 
            .countWindow(100, 10) //每 10 个元素统计过去 100 个元素的数量之和
            .sum(1);
        ```

#### 三. TIme
上现在介绍window时, 有一种window时基于时间的, 那这个时间到底指的是什么时间?
Flink 在流程序中支持不同的 Time 概念，就比如有 Processing Time、Event Time 和 Ingestion Time。

1. **processing time**  
   是指事件被处理时机器的系统时间。
2. **event time**  
   Event Time 是事件发生的时间，一般就是数据本身携带的时间。
3. **ingestion time**  
   Ingestion Time 是事件进入 Flink 的时间。


区别:   
1. 在分布式和异步的环境下，`Processing Time` 不能提供确定性，因为它容易受到事件到达系统的速度（例如从消息队列）、事件在系统内操作流动的速度以及中断的影响。
2. 完美的说，无论事件什么时候到达或者其怎么排序，最后处理`Event Time`将产生完全一致和确定的结果。但是，除非事件按照已知顺序（按照事件的时间）到达，否则处理 Event Time 时将会因为要等待一些无序事件而产生一些延迟。
3. `Ingestion Time` 在概念上位于 Event Time 和 Processing Time 之间。 与 Processing Time 相比，它稍微贵一些，但结果更可预测。因为 Ingestion Time 使用稳定的时间戳（在源处分配一次）. 与 Event Time 相比，Ingestion Time 程序无法处理任何无序事件或延迟数据，但程序不必指定如何生成水印。在 Flink 中，Ingestion Time 与 Event Time 非常相似，但 Ingestion Time 具有自动分配时间戳和自动生成水印功能。

![img](https://mmbiz.qpic.cn/mmbiz_jpg/1flHOHZw6RubCwphkOInSxCPp7ShaGQkBRyEUaqx0uic4sePsAEjIufnrk9Dq4kLdw04Ela0AgO0ZVPko9SjG0Q/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

下面的代码, 展示了: Flink DataStream 程序的第一部分通常是设置基本时间特性
```java
final StreamExecutionEnvironment env = StreamExecutionEnvironment.getExecutionEnvironment();
 env.setStreamTimeCharacteristic(TimeCharacteristic.ProcessingTime);
 
 // 其他
 // env.setStreamTimeCharacteristic(TimeCharacteristic.IngestionTime);
 // env.setStreamTimeCharacteristic(TimeCharacteristic.EventTime);
 
 DataStream<MyEvent> stream = env.addSource(new FlinkKafkaConsumer09<MyEvent>(topic, schema, props));

stream
    .keyBy( (event) -> event.getUser() )
    .timeWindow(Time.hours(1))
    .reduce( (a, b) -> a.add(b) )
    .addSink(...);
```

#### 四. Event Time 和 Watermarks(水印)   
上面提到, 3中time中有一种是事件时间(Event Time), watermarks就是衡量事件时间进度的一个方法 , Watermark（t）声明 Event Time 已到达该流中的时间 t，这意味着流中不应再有具有时间戳 t'<= t 的元素（即时间戳大于或等于水印的事件）  
Flink 实现了数据流模型中的许多技术。有关 Event Time 和 Watermarks 的详细介绍，请查看以下文章：
* https://www.oreilly.com/ideas/the-world-beyond-batch-streaming-101
* https://research.google.com/pubs/archive/43864.pdf


#### 五. 并行度与slot
1. parallelism  
   parallelism 是并行的意思，在 Flink 里面代表每个任务的并行度，适当的提高并行度可以大大提高 job 的执行效率，比如你的 job 消费 kafka 数据过慢，适当调大可能就消费正常了。
    ```java
    // 设置并行度
    StreamExecutionEnvironment env = StreamExecutionEnvironment.getExecutionEnvironment();
    env.setParallelism(10);
    ```
2. 什么是 slot？  
    slot 在 Flink 里面可以认为是资源组，Flink 将每个任务分成子任务并且将这些子任务分配到 slot 来并行执行程序。如下图, flink job的运行主要是通过`Task Manager`从 `Job Manager`处接收需要部署的`Task`，任务的并行性由每个`Task Manager`上可用的`slot`决定。
    ![img](https://mmbiz.qpic.cn/mmbiz_jpg/1flHOHZw6Rvj6lZzxLztQA5Nyf101WmibyxZXEsdBrD6QmPXgPkCmWc9h6HXhvr28fnMOQ0jPpiaicU0xCvczPnXg/640?wx_fmt=jpeg&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

    例如，如果 Task Manager 有四个 slot，那么它将为每个 slot 分配 25％ 的内存。 可以在一个 slot 中运行一个或多个线程。 同一 slot 中的线程共享相同的 JVM。 同一 JVM 中的任务共享 TCP 连接和心跳消息。Task Manager 的一个 Slot 代表一个可用线程，该线程具有固定的内存，注意 Slot 只对内存隔离，没有对 CPU 隔离。默认情况下，Flink 允许子任务共享 Slot，即使它们是不同 task 的 subtask，只要它们来自相同的 job。这种共享可以有更好的资源利用率。


#### 六. Flink状态管理和容错机制介绍
1. 什么是有状态的计算  
   计算任务的结果不仅仅依赖于输入，还依赖于它的当前状态，其实大多数的计算都是有状态的计算。比如wordcount,给一些word,其计算它的count,这是一个很常见的业务场景。count做为输出，在计算的过程中要不断的把输入累加到count上去，那么count就是一个state。

2. 批处理和storm中缺少对状态的管理   
    在传统的批处理中，数据是划分为块分片去完成的，然后每一个Task去处理一个分片。当分片执行完成后，把输出聚合起来就是最终的结果。在这个过程当中，对于state的需求还是比较小的。     
    对于流计算而言，对State有非常高的要求，因为在流系统中输入是一个无限制的流，会运行很长一段时间，甚至运行几天或者几个月都不会停机。在这个过程当中，就需要将状态数据很好的管理起来。很不幸的是，在传统的流计算系统中，对状态管理支持并不是很完善。比如storm,没有任何程序状态的支持，一种可选的方案是storm+hbase这样的方式去实现，把这状态数据存放在Hbase中，计算的时候再次从Hbase读取状态数据，做更新在写入进去。

3. Flink中的状态管理  
    * 状态的存储和访问
    * 状态的备份和回复
    * 状态的划分和动态扩展
      * Keyed States
      * Operator State

4. 使用Checkpoint提高程序的可靠性    
    * 用户可以根据的程序里面的配置将checkpoint打开，给定一个时间间隔后，框架会按照时间间隔给程序的状态进行备份。当发生故障时，Flink会将所有Task的状态一起恢复到Checkpoint的状态。从哪个位置开始重新执行。
    * Flink也提供了多种正确性的保障，包括：
      * AT LEAST ONCE;
      * Exactly once;
    
5. 从已停止作业的运行状态中恢复  
    当组件升级的时候，需要停止当前作业。这个时候需要从之前停止的作业当中恢复，Flink提供了2种机制恢复作业:
     * `Savepoint`:是一种特殊的checkpoint，只不过不像checkpoint定期的从系统中去触发的，它是用户通过命令触发，存储格式和checkpoint也是不相同的，会将数据按照一个标准的格式存储，不管配置什么样，Flink都会从这个checkpoint恢复，是用来做版本升级一个非常好的工具；
    * `External Checkpoint`：对已有checkpoint的一种扩展，就是说做完一次内部的一次Checkpoint后，还会在用户给定的一个目录中，多存储一份checkpoint的数据；

6. 状态管理和容错机制实现  
   Flink提供了3种不同的StateBackend
    * `MemoryStateBackend`
    * `FsStateBackend`
    * `RockDBStateBackend`  

   用户可以根据自己的需求选择，如果数据量较小，可以存放到MemoryStateBackend和FsStateBackend中，如果数据量较大，可以放到RockDB中。

 
 #### 七. 二阶段提交-提供Exactly onece语义
 1. 什么是二阶段提交   
     2PC将分布式事务分成了两个阶段: 预提交（投票）和提交（执行）。协调者根据参与者的响应来决定是否需要真正地执行事务，具体流程如下。
     * 预提交阶段: 
        1. 首先 : 协调者向所有参与者发送prepare请求和事务的内容;
        2. 然后 : 参与者在收到请求后, 执行事务中的内容, 并记录redo(用于重放)和undo(用于回滚)日志, 但参与者并不会真正提交事务
        3. 最后 : 所有参与者向协调者返回实务操作预执行的结果: yes或no
     * 提交阶段: 
        提交阶段, 需要协调者判断是否所有参与者在预提交阶段都返回了yes: 
        * 若所有参与者都返回yes:  
            1. 首先, 协调者向所有参与者发送commit请求
            2. 然后, 所有参与者受到commit请求后, 在本机执行真正的事务提交动作, 事务占用的事务资源, 冰箱协调者返回ack
            3. 最后, 协调者受到所有参与者的ack消息, 事务成功执行
        * 若存在参与者返回no, 或有的参与者超市未返回: 
            1. 首先, 协调者向所有参与者发送回滚请求(rollback)
            2. 然后, 所有参收到rollback请求后, 根据undo日志回滚事务到执行动作之前的状态, 释放占用的事务资源. 并向者向协调者返回ack
            3. 最后, 协调者受到所有ack, 事务回滚完成  
2. 2PC的优缺点  
   2PC的优点在于原理非常简单，容易理解及实现。缺点主要有3个，列举如下：
    1. 协调者存在单点问题。如果协调者挂了，整个2PC逻辑就彻底不能运行。
    2. 执行过程是完全同步的。所有参与者在等待其他参与者响应的过程中处于阻塞状态，不能继续执行任务, 大并发下有性能问题。  
    3. 仍然存在不一致风险。如果由于网络异常等意外导致只有部分参与者收到了commit请求，就会造成部分参与者提交了事务而其他参与者未提交的情况。  

3. flink依靠checkpoint和分布式快照算法保证`exactly once`语义  
    Flink中提供了基于2PC的SinkFunction，名为`TwoPhaseCommitSinkFunction`，帮助我们做了一些基础的工作。官方推荐所有需要保证exactly once的Sink逻辑都继承该抽象类。它定义了如下4个抽象方法，需要子类实现。
    ```java
    protected abstract TXN  beginTransaction() throws Exception;
    protected abstract void preCommit(TXN transaction) throws Exception;
    protected abstract void commit(TXN transaction);
    protected abstract void abort(TXN transaction);
    ```
    * `beginTransaction()` ：开始一个事务，返回事务信息的句柄。
    * `preCommit()`：预提交（即提交请求）阶段的逻辑。
    * `commit()`：正式提交阶段的逻辑。
    * `abort()`：取消事务。   
  
   在继续之前，先看下对 checkpoint 机制的简要介绍, 一次 checkpoint 是以下内容的一致性快照：
    * 应用程序的`当前状态`
    * `输入流的位置`  
   
   checkpoint是定期执行的, 数据可写入外部存储系统, 例如: HDFS. Checkpoint写入外部存储是异步执行的, 意味着flink在进行checkpoint时可以继续处理数据. 如果发生机器或软件故障，重新启动后，Flink 应用程序将从最新的 checkpoint 点恢复处理； Flink 会恢复应用程序状态，将输入流回滚到上次 checkpoint 保存的位置，然后重新开始运行。这意味着 Flink 可以像从未发生过故障一样计算结果。为了提供端到端的 Exactly-Once 语义, 除了 Flink 应用， Flink 写入的外部系统也需要能满足 Exactly-Once 语义, 这些外部系统必须提供提交或回滚的方法，然后通过 Flink 的 checkpoint 机制来协调。

4. 如何使用checkponit实现状态保存(保存状态和数据源的offset)的`exactly once`语义
   1. 预提交阶段:  
       在checkpoint开始的时候, 即"预提交阶段", Flink的`JobManager`会将`checkpoint barrier`注入数据流, 将数据流中的记录分为进入当前checkpoint和进入下一个checkpoint. 对于每一个收到barrier的operator, 将状态快照写入到`state backend`(内存,文件或rockdb). 这种方式仅适用于 operator 具有`内部状态`。所谓内部状态，是指`Flink statebackend`保存和管理的 - 例如，第二个 operator 中 window 聚合算出来的 sum 值。当一个进程有它的内部状态的时候，除了在 checkpoint 之前需要将数据变更写入到 state backend ，不需要在预提交阶段执行任何其他操作。Flink 负责在 checkpoint 成功的情况下正确提交这些写入，或者在出现故障时中止这些写入。
   2. 提交阶段  
        下一步是通知所有 operator，checkpoint 已经成功了。这是两阶段提交协议的提交阶段，JobManager 为应用程序中的每个 operator 发出 checkpoint 已完成的回调。
   3. 提交外部事务  
        数据输出端（Data Sink）拥有外部状态，此时应该提交外部事务。

